In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
from epi.normalizing_flows import NormalizingFlow
import tensorflow as tf
import tensorflow_probability as tfp
tfb = tfp.bijectors
tfd = tfp.distributions

In [3]:
def trainable_lu_factorization(event_size, batch_shape=(), seed=None, dtype=tf.float32, name=None):
    with tf.name_scope(name or 'trainable_lu_factorization'):
        event_size = tf.convert_to_tensor(event_size, dtype_hint=tf.int32, name='event_size')
        batch_shape = tf.convert_to_tensor(batch_shape, dtype_hint=event_size.dtype, name='batch_shape')
        random_matrix = tf.random.uniform(
            shape=tf.concat([batch_shape, [event_size, event_size]], axis=0),
            dtype=dtype,
            seed=seed)
        random_orthonormal = tf.linalg.qr(random_matrix)[0]
        lower_upper, permutation = tf.linalg.lu(random_orthonormal)
        lower_upper = tf.Variable(
            initial_value=lower_upper,
            trainable=True,
            name='lower_upper')
        # Initialize a non-trainable variable for the permutation indices so
        # that its value isn't re-sampled from run-to-run.
        permutation = tf.Variable(
            initial_value=permutation,
            trainable=False,
            name='permutation')
    return lower_upper, permutation

In [4]:
lower_upper, perm = trainable_lu_factorization(4)
print(lower_upper)
print(perm)

<tf.Variable 'trainable_lu_factorization/lower_upper:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.59904504,  0.52042955,  0.6082792 , -0.01715939],
       [ 0.89346045, -1.1430819 , -0.5045728 , -0.486869  ],
       [ 0.9515199 ,  0.23153283, -1.2138451 ,  0.36701193],
       [ 0.28806114,  0.53790987, -0.28628635,  1.2030947 ]],
      dtype=float32)>
<tf.Variable 'trainable_lu_factorization/permutation:0' shape=(4,) dtype=int32, numpy=array([0, 2, 3, 1], dtype=int32)>


In [6]:
bij = tfp.bijectors.MatvecLU(
    lower_upper, perm, validate_args=False, name=None
)

Instructions for updating:
`MatvecLU` has been deprecated and renamed `ScaleMatvecLU`; please use that symbol instead.


In [16]:
x = np.random.normal(0., 1., (4,)).astype(np.float32)
print(bij(x))
print(bij.forward_log_det_jacobian(x, event_ndims=1))

tf.Tensor([-0.9541798 -0.6462333  0.4466392 -0.8013915], shape=(4,), dtype=float32)
tf.Tensor(-1.937151e-07, shape=(), dtype=float32)


In [8]:
D = 2
#lb = -1*np.ones((D,))
#ub = 1*np.ones((D,))
nf = NormalizingFlow(
    "coupling", D, 2, 2, 25, batch_norm=False,
    num_bins=32, elemwise_fn="spline",
    post_affine=True, random_seed=1,
    #post_affine=True, bounds=(lb,ub), random_seed=1,
)
print(nf)

In [3]:
z, log_q_z = nf(10)

In [5]:
print(nf.bijector_fns)

ListWrapper([(), <epi.normalizing_flows.SplineParams object at 0x7fa631ecaef0>, (), <epi.normalizing_flows.SplineParams object at 0x7fa62e3fd518>])


In [4]:
for param in nf.trainable_variables:
    print(param)

<tf.Variable 'real_nvp/forward_log_det_jacobian/w/kernel:0' shape=(1, 32) dtype=float32, numpy=
array([[ 0.4226386 ,  0.22016853,  0.08234906,  0.02216202, -0.3641784 ,
         0.08509195, -0.22607428, -0.38135204, -0.20102943,  0.25581813,
         0.25211662,  0.33974403,  0.1887958 ,  0.37081164, -0.348355  ,
        -0.08980599, -0.21058312,  0.08387399,  0.41272944, -0.1485582 ,
        -0.30277687,  0.2192108 , -0.3595749 ,  0.01056543, -0.10300162,
        -0.34355605,  0.17857444,  0.4082812 , -0.07055354, -0.04112777,
        -0.15531081,  0.22308838]], dtype=float32)>
<tf.Variable 'real_nvp/forward_log_det_jacobian/w/bias:0' shape=(32,) dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>
<tf.Variable 'real_nvp/forward_log_det_jacobian/h/kernel:0' shape=(1, 32) dtype=float32, numpy=
array([[-0.24377292,  0.09346706, -0.03335997, -0.1520679 ,  

Need to broadcast spline parameters over batch dimension as well.
"Reshaping acrobatics needed."

In [2]:
class SplineParams(tf.Module):

    def __init__(self, nunits, nbins=32):
        self._nunits = nunits
        self._nbins = nbins
        self._built = False
        self._bin_widths = None
        self._bin_heights = None
        self._knot_slopes = None

    def _bin_positions(self, x):
        x = tf.reshape(x, [-1, self._nunits, self._nbins])
        return tf.math.softmax(x, axis=-1) * (2 - self._nbins * 1e-2) + 1e-2

    def _slopes(self, x):
        x = tf.reshape(x, [-1, self._nunits, self._nbins - 1])
        return tf.math.softplus(x) + 1e-2

    def __call__(self, x, nunits):
        if not self._built:
            self._bin_widths = tf.keras.layers.Dense(
              nunits * self._nbins, activation=self._bin_positions, name='w')
            self._bin_heights = tf.keras.layers.Dense(
              nunits * self._nbins, activation=self._bin_positions, name='h')
            self._knot_slopes = tf.keras.layers.Dense(
              nunits * (self._nbins - 1), activation=self._slopes, name='s')
            self._built = True
        return tfb.RationalQuadraticSpline(
            bin_widths=self._bin_widths(x),
            bin_heights=self._bin_heights(x),
            knot_slopes=self._knot_slopes(x))
    
N = 100 # batch size
D = 15 # dimensionality
nsplits = 3

xs = np.random.randn(N, D).astype(np.float32)  # Keras won't Dense(.)(vec).
nmasked = [5*i for i in range(nsplits)] # dimensions to mask in RealNVP
nunits = [D - x for x in nmasked]
splines = [SplineParams(nunits[i]) for i in range(nsplits)]

def spline_flow():
    stack = tfb.Identity()
    for i in range(nsplits):
        stack = tfb.RealNVP(nmasked[i], bijector_fn=splines[i])(stack)
    return stack

ys = spline_flow().forward(xs)
ys_inv = spline_flow().inverse(ys)  # ys_inv ~= xs
assert(np.isclose(xs, ys_inv).all())

In [7]:
N = 100 # batch size
D = 15 # dimensionality
nsplits = 3

xs = np.random.randn(N, D).astype(np.float32)  # Keras won't Dense(.)(vec).
nmasked = [5*i for i in range(nsplits)] # dimensions to mask in RealNVP
nunits = [D - x for x in nmasked]
splines = [SplineParams(nunits[i]) for i in range(nsplits)]

def spline_flow():
    stack = tfb.Identity()
    for i in range(nsplits):
        stack = tfb.RealNVP(nmasked[i], bijector_fn=splines[i])(stack)
    return stack

ys = spline_flow().forward(xs)
ys_inv = spline_flow().inverse(ys)  # ys_inv ~= xs
assert(np.isclose(xs, ys_inv).all())

In [3]:
D = 15
num_masked = 7
xs = np.random.randn(1, D).astype(np.float32)  # Keras won't Dense(.)(vec).
nunits = D - num_masked
spline = SplineParams(nunits)

#def spline_flow():
nvp = tfb.RealNVP(num_masked, bijector_fn=spline)
#return stack

#ys = spline_flow().forward(xs)
#ys_inv = spline_flow().inverse(ys)  # ys_inv ~= xs

In [16]:
s = splines[1]
print(s)

In [20]:
f = s(xs[:,:5], 5)

In [24]:
print(f)
print(f(xs[:,:10]))

tf.Tensor(
[[-1.3599697   0.1139448   0.8640732  -0.82076377 -2.0931187  -0.5929633
   0.48693687  0.7457276   0.43292007 -0.56522083]], shape=(1, 10), dtype=float32)


In [6]:
num_layers = 2
num_units = 25
f = tfb.real_nvp_default_template(
    hidden_layers=num_layers * [num_units])

In [7]:
f = tfb.RationalQuadraticSpline(
        bin_widths=self._bin_widths(x),
        bin_heights=self._bin_heights(x),
        knot_slopes=self._knot_slopes(x))
print(f)

In [21]:
M = 10
D = 4
x = np.random.normal(0., 1., (M,D))

In [22]:
a, b = f(x, 2)
print(a.shape, b.shape)
print(a[0,:], b[0,:])

(10, 2) (10, 2)
tf.Tensor([-0.0893935   0.32305841], shape=(2,), dtype=float64) tf.Tensor([-0.22781825  0.53631505], shape=(2,), dtype=float64)


In [3]:
D = 2
lb = -1*np.ones((D,))
ub = 1*np.ones((D,))
nf = NormalizingFlow(
    "coupling", D, 2, 2, 25, batch_norm=True,
    post_affine=True, bounds=(lb,ub), random_seed=1,
)
mu = .75*np.ones((D,))
Sigma = .25*np.eye(D)
#nf.initialize(mu, Sigma, num_iters=int(10e3), verbose=True)